In [1]:
import numpy as np
import gym
import torch
from torch import nn
import torch.nn.functional as F
import copy
from time import time

In [2]:
torch.set_grad_enabled(False)

## Neuroevolution Setup

In [3]:
class Agent(nn.Module):
    '''The brain of the agent'''
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(nn.Linear(4, 32),
                                nn.ReLU(),
                                nn.Linear(32, 2))
        
    def forward(self, inputs):
        x = self.fc(inputs)
        return F.softmax(x, dim=1)

In [4]:
def initialize_population(pop_size=2):
    '''Randomly initialize a bunch of agents'''
    population = [Agent() for _ in range(pop_size)]
    
    return population

In [5]:
def evaluate_agent(agent, episodes=15, max_episode_length=250):
    '''
    한 agent를 한번만 돌리는게 아니라 'episoides'수만큼 돌린 후 나온 reward들의 평균을 그 agent의 reward로 봄
    예를 들어 한 세대의 agent를 3개(agent1,agent2,agent3)로 설정하고 episodes를 5로 설정하면
    agent1-1, agent1-2, agent1-3, agent1-4, agent1-5의 reward들의 평균이 agent1의 최종 reward가 되는 것

    max_episode_length는 얻을수있는 최대 reward를 의미
    매 스텝(카트의 좌우움직임 한번)마다 reward가 +1이 됨
    즉 설정한 max_episode_length만큼 스텝하는동안 막대를 안떨어뜨렸으면 그 episode의 reward는 max_episode_length와 같음 '''
    env = gym.make("CartPole-v0")
    agent.eval()
    
    total_rewards = []
    
    for ep in range(episodes):
        observation = env.reset()
        # Modify the maximum steps that can be taken in a single episode
        env._max_episode_steps = max_episode_length
        
        episodic_reward = 0
        # Start episode
        for step in range(max_episode_length):
            #env.render()    #렌더링 없애고 싶으면 이부분 주석처리
            input_obs = torch.Tensor(observation).unsqueeze(0)
            observation, reward, done, info = env.step(agent(input_obs).argmax(dim=1).item())
            
            episodic_reward += reward
            if done:
                break
                
        total_rewards.append(episodic_reward)
        
                
    return np.array(total_rewards).mean()

In [6]:
def evaluate_population(population, episodes=15, max_episode_length=250):
    '''Evaluate the population'''
    pop_fitness = []
    
    for agent in population:
        pop_fitness.append(evaluate_agent(agent, episodes, max_episode_length))
        
    return pop_fitness

In [7]:
'''
처음에 topology를 고정했기 때문에 crossover는 하지 않고 바로 mutation

mutation방법: 부모 agnet의 parameter에 정규분포에서 뽑은 랜덤한 수 x mutation_power를 더해줌
'''

def mutate(parent_agent, mutation_power=0.02):
    child_agent = copy.deepcopy(parent_agent)
    
    for param in child_agent.parameters():
        param.data = param.data + (torch.randn(param.shape) * mutation_power)
        
    return child_agent

In [8]:
def repopulate(top_agents, pop_size, mutation_power):
    '''Repopulate the population from the top agents by mutation'''
    new_population = []
    
    n = 0
    while(n < pop_size):
        for parent in top_agents:
            child = mutate(parent, mutation_power)
            new_population.append(child)
            n += 1
            
    return new_population[:pop_size - 1]

## Train

In [9]:
TRAINED_AGENT = {}

In [16]:
'''
정해진 타임만큼 학습할건지 정해진 세대만큼 학습할건지 정할수 있음 (아래 주석 참고)
'''

def evolve(generations=10, max_time=80, 
           pop_size=100, 
           topK=20, 
           episodes=15, 
           max_episode_length=250, 
           mutation_power=0.02):
    '''
    topK: 자손세대를 만들 부모세대 agent수
    예를 들어 pop_size=100, topK=20이면 부모세대의 agent 100개 중 reward가 가장 높은 20개의 agent들로 자식세대 agent100개를 만듦
    '''
    
    global TRAINED_AGENT
    
    population = initialize_population(pop_size)
    global_best = {}
    
    t1 = time()
    #g = 0 # 정해진 타임만큼 돌리고 싶을때 
    for g in range(generations):      # 정해진 세대만큼 돌릴 때
    #while ((time() - t1) <= max_time): # 정해진 타임만큼 돌리고 싶을때 
        
        # Evaluate the population
        pop_fitness = evaluate_population(population, episodes, max_episode_length)
        mean_pop_reward = np.array(pop_fitness).mean()
        
        # Rank the agents in descending order
        topK_idx = np.argsort(pop_fitness)[::-1][:topK]
        topK_agents = [population[i] for i in topK_idx]
        
        # Get Best Agent
        best_agent = population[topK_idx[0]]
        best_reward = pop_fitness[topK_idx[0]]
        
        # Check with global best
        if g == 0:
            global_best['reward'] = best_reward
            global_best['agent'] = best_agent
        else:
            if best_reward >= global_best['reward']:
                global_best['reward'] = best_reward
                global_best['agent'] = best_agent
                
        print('Generation', g)
        print('Mean Reward of Population', mean_pop_reward)
        print('Best Agent Reward (mean)', best_reward)
        print('Global Best Reward (mean)', global_best['reward'], '\n')
        
        # Mutate and Repopulate
        new_population = repopulate(topK_agents, pop_size, mutation_power)
        # take the best agent of generation forward without cloning as well
        new_population.append(best_agent)
        
        population = new_population
        
        TRAINED_AGENT = global_best
        
        #g += 1 # 정해진 타임만큼 돌리고 싶을때 

In [19]:
evolve(generations=20,
       pop_size=20, 
       topK=10, 
       episodes=15, 
       max_episode_length=200, 
       mutation_power=0.02)
#총 렌더링 수: generations x pop_size x episodes

Generation 0
Mean Reward of Population 14.703333333333337
Best Agent Reward (mean) 65.4
Global Best Reward (mean) 65.4 



## Test the Trained Agent

In [14]:
def play_agent(agent, episodes=5, max_episode_length=200, render=False):
    env = gym.make("CartPole-v0")
    
    agent.eval()
    
    total_rewards = []
    
    for ep in range(episodes):
        observation = env.reset()
        env._max_episode_steps = max_episode_length
        
        episodic_reward = 0
        
        for step in range(max_episode_length):
            if render:
                env.render()
            
            input_obs = torch.Tensor(observation).unsqueeze(0)
            observation, reward, done, info = env.step(agent(input_obs).argmax(dim=1).item())
            
            episodic_reward += reward
            if done:
                break
        
        
        total_rewards.append(episodic_reward)
    
    env.close()
    print('Mean Rewards across all episodes', np.array(total_rewards).mean())
    print('Best Reward in any single episode', max(total_rewards))

In [20]:
'''위에 evolve함수로 학습한 것들 중 가장 마지막 세대의 cartpole학습을 볼 수 있음'''
play_agent(TRAINED_AGENT['agent'], episodes=5, max_episode_length=200, render=True)

Mean Rewards across all episodes 59.6
Best Reward in any single episode 70.0


In [ ]:
torch.save(TRAINED_AGENT['agent'].state_dict(), 'model-200.pth')